# Radiative Electric Field due to Oscillating Charge

## Introduction

A static point-charge creates an electric field in all points in space. These field lines point radially outwards and decreases with a $\frac{1}{r^2}$ dependency. What happens when you introduce a velocity or even an acceleration to this point-charge? The fields radiated by the point-charged become much more complex and can usually be described using electrodyanmical laws. 

When a point charge begins to accelerate, a special field called radiation transports energy irreversibly out to infinity. Technically, a change in electric fields creates new magnetic fields, which in turn, the new magnetic fields create new electric fields. This takes the form of electro-magnetic waves described by Maxwell's equations, but for our purposes, we will focus on the radiative electric field only. 

## Goal
The goal of this program is to explore how a radiative electric field due to a point-charge evolves over time. Specifically, treating the point-charge as an oscillator.

## Theory

At an instantaneous point in time $t$, the electric radiative field can be calculated using:
$$ \vec{E_{rad}} = \frac{1}{4 \pi \epsilon_0} \frac{q \vec{a_{perp}}}{c^2 r} $$

where $\epsilon_0$ is a pearmittivity of free space, $q$ is the charge, $c$ is the speed of light, $r$ is the distance from point charge and observation location, and $\vec{a_{perp}}$ is the acceleration that is perpendicular to $\vec{r}$. This is a pretty trivial field to calculate since there is nothing changing.

To create a system in which the radiation evolves over time, the source (point-charge) must also evolve over time. Suppose we have a hydrogen atom that has one electron bonded to the nucleus. Classically, we can model the bond between the electron and hydrogen nucleus as a mass on a spring. The time evolution solutions for a simple harmonic oscillator is given by: 

$$ y(t) = A \;cos(\omega t) $$
$$ v_y(t) = - A \omega sin(\omega t)$$
$$ a_y(t) = - A \omega^2 cos(\omega t) $$

where $y(t)$ is the y position of the oscillating mass, $v_y(t)$ is the speed in the y-direction, $a_y(t)$ is the acceleration in the y-direction, and  $\omega = \sqrt{\frac{k}{m}}$. For our case, $k$ is the bond stifness between the hydrogen nucleus and the electron, and $m$ is the mass of the electron. Now that we have something driving our point-charge, the radiative electric field will begin to evovle over time.

There is one more issue that arises when trying to simulate the time-evolution of $\vec{E_{rad}}$. In reality, there is a time delay between the observation points close to the particle and point farther away from the particle. Therefore, we must adjust our analytical solution for $\vec{E_{rad}}$. 

To begin, lets start by generalizing $\vec{a_{perp}}$ in terms of the observation location $\vec{r}$ and the full acceleraction vector $\vec{A}$ We can re-write $\vec{a_{perp}}$ as:

$$ \vec{a_{perp}} =  ((\vec{A_{max}} \cdot \hat{r})\hat{r} -  \vec{A_{max}}) $$

Next we still need to include some sort of time evolution to the  $\vec{E_{rad}}$ solution. Fortunately we know that the radiative electric field depends on the acceleration. Since we already have a predefined solution for acceleration, $a_y(t)$, which depends on $cos(\omega t) $, we can include this the general solution to evolve it over time. For this to be mathmatically true, $\vec{A_{max}}$ must equal the maximum amplitude ($ A \omega^2$), times a unit vector of $(0,1,0)$. The cosine will take care of the diminishing effects in the new $\vec{E_{rad}}$ solution.

We still need to cover the time delay, luckily, we can take advantage of the cosine and add a phase shift that depends on distance. Since radiation travels at the speed of light $c$, then the time-phase shift will end up being $\frac{r}{c}$.

Finally, our analytical solution of a radiative electric field is:

$$ \vec{E_{rad}} = \frac{1}{4 \pi \epsilon_0} \frac{q}{c^2 r}  ((\vec{A_{max}} \cdot \hat{r})\hat{r} -  \vec{A_{max}}) \; cos(\omega (t - \frac{r}{c}))  $$

## Simulation 

This program will visualize the time-evolution of a radiative electric field due to an oscillating electron. 

### Assupmptions

- The equations used assume propagation in a vacuum
- Ball-Spring Model of Electron and Hydrogen Nucleus
- $k = \frac{q^2}{\alpha}$, where $\alpha$ is the polarizability of hydrogen
- For visualization purposes, electron and rad fields are scaled accordingly
- Magnetic and Electric velocity fields are negligible

In [1]:
import numpy as np
import matplotlib.pyplot
#animate with vpython
from vpython import * 


<IPython.core.display.Javascript object>

The 'Erad' function creates multiple observation points using spherical coordinates. It calculates the radiative electric fields at the created observation points. Displays an animation of the time evolution for a given set of 'numTheta' and 'numPhi' parameters. These parameters dictate how many sets of observation locations the user wants in the theta and phi directions.

In [2]:
def Erad(numTheta, numPhi):
    # Constants
    scene = canvas(title="Erad due to Oscillating Electron")

    q = 1.6e-19 #C
    alpha = 7.4e-41
    e_0 = 8.85e-12
    mu_0 = 4*np.pi*10**-7
    R = 2.82e-15
    c = 3e8
    a = 5.29e-11 #Bohr Radius
    E_0 = 13.6 
    k = q**2 / alpha
    m = 9.11e-31
    A = 4*a
    oofpez = 9e9

    omega = np.sqrt(k/m) #*10**-25

    h = 10**-18
    Nsteps = 1000 #N steps for T seconds of evolution

    t = np.arange(0,Nsteps*h,h)
    y = A*np.cos(omega*t)
    Ay = -A*(omega**2)
    Avec = Ay*vec(0,1,0)

    rsource = []

    for i in range(np.size(y)):
        rsource.append(vec(0,y[i],0))


    robs =  []
    Efields = []
    Erad = []
    radiation = []

    N = 30
    x = A*5*10**11
    L = 2*x
    dx = L/N

    theta = -np.pi/2
    phi = 0

    dtheta = pi/numTheta
    dphi = 2*pi/numPhi

    ## Creates observation locations using spherical coords.
    while theta < np.pi/2:
        phi = 0
        while phi < 2*np.pi:
            x = A*5*10**11
            for n in range(0,N):
                robs.append(x*vec(np.cos(phi)*np.cos(theta),np.sin(theta),np.sin(phi)*np.cos(theta)))
                x = x + dx
            phi = phi + dphi
        theta = theta + dtheta

    RadNum = np.size(robs)
    ## Erad size [Nsteps,RadNum] [time,robs]

    print("Loading...")

    ## Calculates Erad for each time step and each observation location
    for n in range(0,RadNum):
        ## distance is farther from amplitude of electron
        r = robs[n]
        Efields = []

        for i in range(0,Nsteps):
            Efield = oofpez * q /(c**2 * mag(r)) * (dot(Avec,hat(r)) * hat(r) - Avec) * np.cos(omega*(t[i] + (mag(r)/c))) *13**6
            Efields.append(Efield)
        Erad.append(Efields)

        rad = arrow(pos = r, axis = Erad[n][0], color=color.yellow)
        radiation.append(rad)

    electron  = sphere(pos = rsource[0]*10**11, radius = A*12**10, color = color.blue)
    ## Animate the evolution of Erad
    scene.pause()
    for i in range(1,Nsteps):
        rate(80)
        electron.pos = rsource[i] *10**11

        for n in range(0,RadNum):
            radiation[n].axis = Erad[n][i]
    return print("Animation Complete!")

The cell below animates $\vec{E_{rad}}$ along the x-axis.

In [3]:
Erad(2,2)

<IPython.core.display.Javascript object>

Loading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Animation Complete!


The cell below animates $\vec{E_{rad}}$ along the x-y plane.

In [4]:
Erad(10,2)

<IPython.core.display.Javascript object>

Loading...
Animation Complete!


The cell below animates $\vec{E_{rad}}$ along the x-z plane.

In [5]:
Erad(2,20)

<IPython.core.display.Javascript object>

Loading...
Animation Complete!


The cell below animates $\vec{E_{rad}}$ along the x-y-z plane.

In [6]:
Erad(4,6)

<IPython.core.display.Javascript object>

Loading...
Animation Complete!


The cell below animates $\vec{E_{rad}}$ along the x-y-z plane with even more observation points..

In [ ]:
Erad(8,20)

## Conclusion 

This project is definitely theory heavy. The program uses a lot of lists and for-loops, which is computationally demanding. Creating a class with specified objects, alongside vectorization methods, will definitely cut down the total runtime and allow for even more observation points. In the future, it would be interesting to model the radiative electric field due to a particle in different types of motion. 